In [8]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models

# Set paths
DATASET_PATH = "Deforestation_Data/train/"

# Image size
IMG_SIZE = (256, 256)


In [9]:
def load_images(folder, label):
    images, labels = [], []
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        try:
            img = load_img(img_path, target_size=IMG_SIZE, color_mode="grayscale")  
            img_array = img_to_array(img) / 255.0  
            images.append(img_array)
            labels.append(label)
        except:
            print(f"Skipping corrupted image: {file}")
    return np.array(images), np.array(labels)

# Load images
deforestation_images, deforestation_labels = load_images(os.path.join(DATASET_PATH, "Deforestation"), 1)
normal_images, normal_labels = load_images(os.path.join(DATASET_PATH, "Normal"), 0)

# Combine dataset
X = np.concatenate([deforestation_images, normal_images], axis=0)
y = np.concatenate([deforestation_labels, normal_labels], axis=0)

# Split into train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Samples: {X_train.shape}, Testing Samples: {X_test.shape}")


Skipping corrupted image: -73.15884,46.38819.jpg
Training Samples: (5038, 256, 256, 1), Testing Samples: (1260, 256, 256, 1)


In [15]:
def build_classifier():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 1)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

classifier = build_classifier()
classifier.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=8)


Epoch 1/5
630/630 ━━━━━━━━━━━━━━━━━━━━ 201s 315ms/step - accuracy: 0.7982 - loss: 0.4376 - val_accuracy: 0.8937 - val_loss: 0.2918
Epoch 2/5
630/630 ━━━━━━━━━━━━━━━━━━━━ 198s 314ms/step - accuracy: 0.9239 - loss: 0.2177 - val_accuracy: 0.9095 - val_loss: 0.2942
Epoch 3/5
630/630 ━━━━━━━━━━━━━━━━━━━━ 196s 310ms/step - accuracy: 0.9294 - loss: 0.1822 - val_accuracy: 0.9278 - val_loss: 0.2043
Epoch 4/5
630/630 ━━━━━━━━━━━━━━━━━━━━ 202s 320ms/step - accuracy: 0.9531 - loss: 0.1372 - val_accuracy: 0.9095 - val_loss: 0.2425
Epoch 5/5
630/630 ━━━━━━━━━━━━━━━━━━━━ 207s 329ms/step - accuracy: 0.9632 - loss: 0.1057 - val_accuracy: 0.9167 - val_loss: 0.2876


In [22]:
classifier.save("classifier_model.h5")


In [12]:
def refine_deforestation_mask(image_path):
    # Load image
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding (to detect clear deforested areas)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)

    # Use Canny edge detection to detect strong boundaries of deforested regions
    edges = cv2.Canny(thresh, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank mask and draw contours
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)

    return img, mask



In [14]:
def predict_deforestation(image_path):
    img = load_img(image_path, target_size=IMG_SIZE, color_mode="grayscale")
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = classifier.predict(img_array)[0][0]

    if prediction > 0.5:
        print("🛑 Deforestation Detected! Identifying affected areas...")

        original, mask = refine_deforestation_mask(image_path)  # Get refined deforestation mask

        # Convert mask to color and overlay it
        mask_colored = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        output = cv2.addWeighted(original, 0.7, mask_colored, 0.3, 0)

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title("Original Image")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.imshow(mask, cmap="gray")
        plt.title("Detected Deforested Area")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        plt.title("Final Output")
        plt.axis("off")

        plt.show()

    else:
        original, mask = refine_deforestation_mask(image_path)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title("Original Image")
        plt.axis("off")
        print("✅ No deforestation detected.")

# Run on a test image
test_image_path = "Deforestation_Data/test/Deforestation/-73.45352,45.585282.jpg"
predict_deforestation(test_image_path)


NameError: name 'classifier' is not defined